## This notebook is designed to run in ArcGIS Pro

#### Change gdb_path to local file path for your machine - all other code will work as written

In [5]:
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
import pandas as pd

#set filepath to reference later - other users should replace this with a local .gdb filepath
gdb_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab3\GIS5571_Lab3.gdb"

In [15]:
#url ends with "&quick_pick=30_d" to ensure it always takes last 30 days of data. All stations included.
ndawn_url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=229&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=228&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d"
    
#pandas can read csv to dataframe directly from url, we skip rows that include extra data
ndawn_df = pd.read_csv(ndawn_url, skiprows=[0,1,2,4])

#remove unnecessary columns
ndawn_df.drop(["Avg Temp Flag", "Max Temp Flag", "Min Temp Flag", "Elevation"], axis = 1, inplace=True)
#test dataframe
ndawn_df

,Station Name,Latitude,Longitude,Year,Month,Day,Max Temp,Min Temp,Avg Temp
0,Ada,47.32119,-96.51406,2024,10,26,58.856,26.762,42.809
1,Ada,47.32119,-96.51406,2024,10,27,60.224,32.099,46.162
2,Ada,47.32119,-96.51406,2024,10,28,65.480,44.130,54.805
3,Ada,47.32119,-96.51406,2024,10,29,58.658,43.349,51.004
4,Ada,47.32119,-96.51406,2024,10,30,46.328,28.666,37.497
...,...,...,...,...,...,...,...,...,...
6527,Zeeland,46.01351,-99.68768,2024,11,20,23.142,14.738,18.940
6528,Zeeland,46.01351,-99.68768,2024,11,21,23.389,7.286,15.338
6529,Zeeland,46.01351,-99.68768,2024,11,22,37.450,7.988,22.719
6530,Zeeland,46.01351,-99.68768,2024,11,23,27.529,21.335,24.432


In [17]:
#Group by stations, get count of how many days per station (some are missing data), get mean for avg, min, and max temp
ndawn_df = ndawn_df.groupby("Station Name").agg({"Latitude": "max", 
                                      "Longitude": "max", 
                                      "Year": "max", 
                                      "Month": "max", 
                                      "Day": "count", 
                                      "Avg Temp": "mean", 
                                      "Max Temp": "mean", 
                                      "Min Temp": "mean"})

#Show table - calculations appear correct and I have the right number of stations (218)
ndawn_df

,Latitude,Longitude,Year,Month,Day,Avg Temp,Max Temp,Min Temp
Station Name,,,,,,,,
Ada,47.32119,-96.51406,2024,11,30,38.420867,45.950033,30.891200
Adams,48.49988,-98.07588,2024,11,30,34.966333,43.145233,26.787067
Alamo,48.54652,-103.47186,2024,11,30,33.207900,44.034033,22.381367
Alexander,47.75056,-103.73358,2024,11,30,35.154700,46.686500,23.622467
Alvarado,48.24594,-97.02153,2024,11,30,37.610400,46.211967,29.008233
...,...,...,...,...,...,...,...,...
Wolford,48.51603,-99.62441,2024,11,30,34.365500,43.805433,24.925233
Wolverton,46.56545,-96.68726,2024,11,30,38.143400,46.071400,30.215100
Ypsilanti,46.77638,-98.52323,2024,11,30,37.488167,46.059700,28.916100


In [20]:
#convert to spatially-enabled dataframe
ndawn_sedf = pd.DataFrame.spatial.from_xy (df = ndawn_df,
                                          x_column = "Longitude",
                                          y_column = "Latitude",
                                          sr = 4326)

ndawn_sedf

,Latitude,Longitude,Year,Month,Day,Avg Temp,Max Temp,Min Temp,SHAPE
Station Name,,,,,,,,,
Ada,47.32119,-96.51406,2024,11,30,38.420867,45.950033,30.891200,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
Adams,48.49988,-98.07588,2024,11,30,34.966333,43.145233,26.787067,"{""spatialReference"": {""wkid"": 4326}, ""x"": -98...."
Alamo,48.54652,-103.47186,2024,11,30,33.207900,44.034033,22.381367,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
Alexander,47.75056,-103.73358,2024,11,30,35.154700,46.686500,23.622467,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
Alvarado,48.24594,-97.02153,2024,11,30,37.610400,46.211967,29.008233,"{""spatialReference"": {""wkid"": 4326}, ""x"": -97...."
...,...,...,...,...,...,...,...,...,...
Wolford,48.51603,-99.62441,2024,11,30,34.365500,43.805433,24.925233,"{""spatialReference"": {""wkid"": 4326}, ""x"": -99...."
Wolverton,46.56545,-96.68726,2024,11,30,38.143400,46.071400,30.215100,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
Ypsilanti,46.77638,-98.52323,2024,11,30,37.488167,46.059700,28.916100,"{""spatialReference"": {""wkid"": 4326}, ""x"": -98...."


In [21]:
ndawn_path = f"{gdb_path}/ndawn_last_30d"

ndawn_sedf.spatial.to_featureclass(location = ndawn_path)

'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab3\\GIS5571_Lab3.gdb\\ndawn_last_30d'

In [14]:
temps_list = ["avg_temp", "min_temp", "max_temp"]

for measure in temps_list:
    arcpy.ddd.Idw(
    in_point_features="ndawn_last_30d",
    z_field=measure,
    out_raster=fr"{gdb_path}\Idw_{measure}_30d",
    cell_size=0.02,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None)
    
    arcpy.ddd.Kriging(
    in_point_features="ndawn_last_30d",
    z_field=measure,
    out_surface_raster=fr"{gdb_path}\Kriging_{measure}_30d",
    semiVariogram_props="Spherical # # # #",
    cell_size=0.02,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None)
    
    arcpy.ga.LocalPolynomialInterpolation(
    in_features="ndawn_last_30d",
    z_field=measure,
    out_ga_layer=None,
    out_raster=fr"{gdb_path}\LPI_{measure}_30d",
    cell_size=0.02,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION")
    

In [15]:
#Set symbology to be the same for each method for avg, min, and max temperatures
for measure in temps_list:
    
    idw_layer = f"Idw_{measure}_30d"
    lpi_layer = f"LPI_{measure}_30d"
    kriging_layer = f"Kriging_{measure}_30d"

    # Apply symbology to LPI layer from the IDW layer
    arcpy.management.ApplySymbologyFromLayer(
        in_layer=lpi_layer,
        in_symbology_layer=idw_layer,
        symbology_fields=None,
        update_symbology="MAINTAIN"
    )
    
    # Apply symbology to Kriging layer from the IDW layer
    arcpy.management.ApplySymbologyFromLayer(
        in_layer=kriging_layer,
        in_symbology_layer=idw_layer,
        symbology_fields=None,
        update_symbology="MAINTAIN"
    )